# Job Management
As you've probably seen in other examples, or in BigQuery in general, nearly everything is expressed in terms of jobs. This is a very common concept in the programming world specially when operations take long times or are distributed and asynchronous.

The design of `BqT` is also based around this concept. For example when you run a query, you can get the job object back without waiting for the results to be ready:

In [1]:
from bqt import bqt

In [2]:
launch_countries = bqt.query_async("""
SELECT *
FROM `subs-analytics.launch_countries.launch_countries` 
WHERE launch_date > '2016-01-03'
""")
launch_countries

As you can see, the call to run a query (and most other calls) return a job result (`BqJobResult` or similar object). This means that we queued the job with BigQuery and this object is a reference to that running job.


## List Running Jobs
Internally, `BqT` keeps refrences to all these jobs, and even more, you can see and cancel jobs that you didn't even run from this notebook:

In [3]:
bqt.list_running_jobs()

,created,destination,elapsed,elapsed_seconds,ended,job_id,query,started,status,this_notebook,user_email,write_disposition
0,2018-06-25 17:32:26.808000+00:00,gro-analytics._55e4bc3d24934ef4c81f7d97d030924...,2 second(s),2.412977,None,6cea4841-a17d-4097-ac98-51ef295714f1,\nSELECT *\nFROM `subs-analytics.launch_countr...,2018-06-25 17:32:27.239000+00:00,running,True,behrooza@spotify.com,WRITE_TRUNCATE


That didn't return anything because you're not running anything in this project. Let's run a few jobs and check again.

In [5]:
for i in range(5):
    bqt.query_async("""
        SELECT *
        FROM `subs-analytics.launch_countries.launch_countries` 
        WHERE launch_date > '2016-01-0{day}'
    """.format(day = i + 1))
    
bqt.list_running_jobs()

,cache_hit,created,destination,elapsed,elapsed_seconds,ended,job_id,query,started,status,this_notebook,write_disposition
0,None,2018-05-29 18:35:44.893000+00:00,wheres-kanye._55e4bc3d24934ef4c81f7d97d0309249...,1 second(s),1.124224,None,4127f457-5f38-4af1-a5aa-2581af63d646,\n SELECT *\n FROM `subs-analyti...,2018-05-29 18:35:45.480000+00:00,running,True,WRITE_TRUNCATE
1,None,2018-05-29 18:35:44.078000+00:00,wheres-kanye._55e4bc3d24934ef4c81f7d97d0309249...,1 second(s),1.939694,None,3fa20164-a5b1-453d-a30e-8be5b67d07aa,\n SELECT *\n FROM `subs-analyti...,2018-05-29 18:35:44.645000+00:00,running,True,WRITE_TRUNCATE
2,None,2018-05-29 18:35:43.450000+00:00,wheres-kanye._55e4bc3d24934ef4c81f7d97d0309249...,2 second(s),2.568010,None,15625cb6-45a8-4728-888a-cc6c95b9cc75,\n SELECT *\n FROM `subs-analyti...,2018-05-29 18:35:43.954000+00:00,running,True,WRITE_TRUNCATE
3,None,2018-05-29 18:35:42.562000+00:00,wheres-kanye._55e4bc3d24934ef4c81f7d97d0309249...,3 second(s),3.456321,None,83e3a482-19f4-47cd-9af3-e0ba706a595c,\n SELECT *\n FROM `subs-analyti...,2018-05-29 18:35:43.146000+00:00,running,True,WRITE_TRUNCATE
4,None,2018-05-29 18:35:41.624000+00:00,wheres-kanye._55e4bc3d24934ef4c81f7d97d0309249...,4 second(s),4.394566,None,24bc1ded-7e02-445e-ac25-a1f9535d7699,\n SELECT *\n FROM `subs-analyti...,2018-05-29 18:35:42.241000+00:00,running,True,WRITE_TRUNCATE


Now we get all 5 queries we just queued. You can even get all running queries in the project, regardless of whether you ran them or not by passing the `all_users=True` parameter:
```python
bqt.list_running_jobs(all_users=True)
```

## Waiting for Jobs to Finish
Let's say here and there you queued 20 jobs and didn't keep track of them (you don't have to unless you need the results). To wait for all of the finish simply:

In [ ]:
bqt.wait_for_all_jobs()

## Tracking an external job in your notebook
Did you run a query in the BigQuery UI but want to use it in your notebook? that's also easy. Just copy the job ID from the BigQuery UI (Go to "Query History", open the query and copy the "Job ID") and:

In [ ]:
job = bqt.add_job_by_id('my_job_id_blah_blah_blah')
job.results  # this will work and will give you a DataFrame directly!

## Cancel a Job
Queued the wrong job? or it's taking too long? you can cancel it both from the BigQuery UI and from your notebook:

In [ ]:
bqt.cancel_job('my_job_id_blah_blah_blah')